# Test of native implementation of TMA

## Import test subjects

In [1]:
import cobra
model = cobra.io.read_sbml_model("iAB_RBC_283.xml")
model.solver = "glpk"

Equilibrator loading hold an important weight on the total time of execution

In [2]:
%%time
import tma

Fragments already downloaded
Fragments already downloaded
CPU times: user 9.05 s, sys: 690 ms, total: 9.74 s
Wall time: 7.68 s


Test whether the compatibility equilibrator_api feature is working properly.

In [3]:
print(f"Number of reactions that aren't transporters: {len(list((filter(lambda x: not tma.is_trans(x), model.reactions))))}")

Number of reactions that aren't transporters: 370


In [4]:
model.metabolites.glc__D_c.annotation

{'bigg.metabolite': 'glc__D',
 'biocyc': 'META:Glucopyranose',
 'chebi': ['CHEBI:12965', 'CHEBI:17634', 'CHEBI:20999', 'CHEBI:4167'],
 'hmdb': ['HMDB00122', 'HMDB06564'],
 'inchi_key': 'WQZGKKKJIJFFOK-GASJEMHNSA-N',
 'kegg.compound': 'C00031',
 'kegg.drug': 'D00009',
 'metanetx.chemical': 'MNXM41',
 'sabiork': ['1406', '1407'],
 'seed.compound': ['cpd00027', 'cpd26821']}

In [5]:
for met in model.metabolites:
    if "inchi_key" in met.annotation:
        met.annotation["inchikey"] = met.annotation["inchi_key"]

In [6]:
from equilibrator_api import compatibility
from equilibrator_cache import create_compound_cache_from_quilt

mapped = compatibility.map_cobra_reactions(
            create_compound_cache_from_quilt(),
            [r for r in model.reactions if not tma.is_trans(r)]
)
mapped

Fragments already downloaded


Incomplete compound stoichiometry in 'GLYOX'.
Incomplete compound stoichiometry in 'GLYt7_211_r'.
Incomplete compound stoichiometry in 'GMPR'.
Incomplete compound stoichiometry in 'GMPS2'.
Incomplete compound stoichiometry in 'GND'.
Incomplete compound stoichiometry in 'GPAM_hs_16_1'.
Incomplete compound stoichiometry in 'GPAM_hs_18_3'.
Incomplete compound stoichiometry in 'GPAM_hs_18_4'.
Incomplete compound stoichiometry in 'GPDDA1'.
Incomplete compound stoichiometry in 'GTHDH'.
Incomplete compound stoichiometry in 'GTHOr'.
Incomplete compound stoichiometry in 'GTHOXti2'.
Incomplete compound stoichiometry in 'GTHPi'.
Incomplete compound stoichiometry in 'GTHS'.
Incomplete compound stoichiometry in 'GUACYC'.
Incomplete compound stoichiometry in 'GUAPRT'.
Incomplete compound stoichiometry in 'GULN3D'.
Incomplete compound stoichiometry in 'GULND'.
Incomplete compound stoichiometry in 'H2O2t'.
Incomplete compound stoichiometry in 'H2Ot'.
Incomplete compound stoichiometry in 'HCO3E'.
Incom

Incomplete compound stoichiometry in 'NRPPHRtu'.
Incomplete compound stoichiometry in 'UREAt'.
Incomplete compound stoichiometry in 'CAT'.
Incomplete compound stoichiometry in 'NT5C'.
Incomplete compound stoichiometry in 'URIt'.
Incomplete compound stoichiometry in 'NTD11'.
Incomplete compound stoichiometry in 'CDIPTr_16_0_16_0'.
Incomplete compound stoichiometry in 'NTD2'.
Incomplete compound stoichiometry in 'XYLK'.
Incomplete compound stoichiometry in 'NMNHYD'.
Incomplete compound stoichiometry in 'XYLTD_D'.
Incomplete compound stoichiometry in 'XYLUR'.
Incomplete compound stoichiometry in 'CDIPTr_16_0_18_1'.
Incomplete compound stoichiometry in 'CDIPTr_16_0_18_2'.
Incomplete compound stoichiometry in 'CDIPTr_18_1_18_1'.
Incomplete compound stoichiometry in 'CDIPTr_18_1_18_2'.
Incomplete compound stoichiometry in 'CDIPTr_18_2_16_0'.
Incomplete compound stoichiometry in 'CDIPTr_18_2_18_1'.
Incomplete compound stoichiometry in 'CDS_16_0_16_0'.
Incomplete compound stoichiometry in 'CDS

Incomplete compound stoichiometry in 'GLYCt'.
Incomplete compound stoichiometry in 'PIPLC_16_0_18_1'.
Incomplete compound stoichiometry in 'PIPLC_16_0_18_2'.
Incomplete compound stoichiometry in 'GLYK'.
Incomplete compound stoichiometry in 'PIPLC_18_1_18_1'.
Incomplete compound stoichiometry in 'PIPLC_18_1_18_2'.
Incomplete compound stoichiometry in 'PIPLC_18_2_16_0'.
Incomplete compound stoichiometry in 'PIPLC_18_2_18_1'.
Incomplete compound stoichiometry in 'PIt'.
Incomplete compound stoichiometry in 'PLA2_2_16_0_16_0'.
Incomplete compound stoichiometry in 'PLA2_2_16_0_18_1'.
Incomplete compound stoichiometry in 'PLA2_2_16_0_18_2'.
Incomplete compound stoichiometry in 'PLA2_2_18_1_18_1'.
Incomplete compound stoichiometry in 'PLA2_2_18_1_18_2'.
Incomplete compound stoichiometry in 'PLA2_2_18_2_16_0'.
Incomplete compound stoichiometry in 'PLA2_2_18_2_18_1'.
Incomplete compound stoichiometry in 'PMANM'.
Incomplete compound stoichiometry in 'PNP'.
Incomplete compound stoichiometry in 'PP

{}

So it isn't.

## Run the test
This test corresponds to a call to `tma()` in the _tma.py_ file.

In [7]:
%%time
normal_solution = model.optimize()

CPU times: user 27.8 ms, sys: 0 ns, total: 27.8 ms
Wall time: 27.5 ms


In [8]:
T = 298.15
RT = T * 8.314472 / 1000
thermo = model.copy()

### 1. eQuilibrator_api side

This function creates the map from equilibrator_api and creates the new variables and constraints of the LP problem.

In [9]:
%%time

thermo = tma.build_tma_problem(model, T)

Fragments already downloaded
CPU times: user 41.5 s, sys: 8.41 s, total: 49.9 s
Wall time: 50.2 s


### 3. Optimize thermodynamic LP problem

In [10]:
%%time
solution = thermo.optimize()

CPU times: user 8.75 ms, sys: 147 µs, total: 8.9 ms
Wall time: 8.56 ms


### 4. Compare solutions of cobrapy and thermo model

In [11]:
print(
        f"Solution of model -> {normal_solution.objective_value}\n"
        f"Sum of thermo fluxes -> {normal_solution.fluxes.sum()}\n"
        f"Solution of thermo model -> {solution.objective_value}\n"
        f"Sum of thermo fluxes -> {solution.fluxes.sum()}"
    )

Solution of model -> 2.935561466666717
Sum of thermo fluxes -> 24.062899866666932
Solution of thermo model -> 2.9355614666667176
Sum of thermo fluxes -> 24.062899866666932
